In [1]:
import rasterio
import numpy as np

In [2]:
VIIRS_INPUT = r"C:\Users\Ankit\Datasets_Forest_fire\updated_viirs_binary_fire_2015_2016.nc"
OUT_FILE = r"C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack.tif"

In [3]:
n_bands = 17544   # hourly for 2015–2016
with rasterio.open(VIIRS_INPUT) as src:
    height, width = src.height, src.width
    src_data = src.read(1)  
    profile = src.profile
    profile.update(
        count=n_bands,
        dtype="uint8",
        driver="GTiff"
    )

    with rasterio.open(OUT_FILE, "w", **profile) as dst:
        for i in range(n_bands):
           
            dst.write(src_data.astype("uint8"), i + 1)

In [4]:
print(f"✅ Created VIIRS time stack with {n_bands} bands → {OUT_FILE}")

✅ Created VIIRS time stack with 17544 bands → C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack.tif


In [1]:
import xarray as xr

In [2]:
ds = xr.open_dataset(r"C:\Users\Ankit\Datasets_Forest_fire\updated_viirs_binary_fire_2015_2016.nc")

In [3]:
ds.data_vars

Data variables:
    fire_label  (time, latitude, longitude) uint8 4MB ...

In [5]:
import numpy as np
np.unique(ds['fire_label'])

array([0, 1], dtype=uint8)

In [6]:
import xarray as xr
import rasterio
from rasterio.transform import from_origin

# Input NetCDF and output GeoTIFF
VIIRS_INPUT = r"C:\Users\Ankit\Datasets_Forest_fire\updated_viirs_binary_fire_2015_2016.nc"
OUT_FILE = r"C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack1.tif"

# Open NetCDF with xarray
ds = xr.open_dataset(VIIRS_INPUT)
fire_label = ds["fire_label"]  # (time, lat, lon)

# Dimensions
n_bands = fire_label.sizes["time"]
height = fire_label.sizes["latitude"]
width = fire_label.sizes["longitude"]

print(f"fire_label shape: {fire_label.shape} → (time, lat, lon)")
print(f"Bands (time steps): {n_bands}, Height: {height}, Width: {width}")

# Build rasterio profile
# Assuming lat/lon are regular grids
lat = fire_label.latitude.values
lon = fire_label.longitude.values
res_lat = abs(lat[1] - lat[0])
res_lon = abs(lon[1] - lon[0])

transform = from_origin(lon.min(), lat.max(), res_lon, res_lat)

profile = {
    "driver": "GTiff",
    "height": height,
    "width": width,
    "count": n_bands,
    "dtype": "uint8",
    "crs": "EPSG:4326",  # NetCDF lat/lon is usually WGS84
    "transform": transform
}

# Write GeoTIFF
with rasterio.open(OUT_FILE, "w", **profile) as dst:
    for i in range(n_bands):
        arr = fire_label.isel(time=i).values.astype("uint8")
        dst.write(arr, i + 1)

    # Add metadata for variable
    dst.update_tags(variable="fire_label", description="Binary fire mask (0/1)")

print(f"✅ Created VIIRS time stack with {n_bands} bands → {OUT_FILE}")


fire_label shape: (17544, 13, 17) → (time, lat, lon)
Bands (time steps): 17544, Height: 13, Width: 17
✅ Created VIIRS time stack with 17544 bands → C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack1.tif
